In [5]:
import tensorflow as tf
from object_detection.utils import dataset_util, config_util
from object_detection.builders import model_builder
import os
from tqdm import tqdm

# 경로 설정
root_dir = 'D:/베리어프리존(장애물 없는 생활공간) 주행영상'
training_tfrecord_file = os.path.join(root_dir, 'Training', 'jpg.tfrecord')
validation_tfrecord_file = os.path.join(root_dir, 'Validation', 'jpg.tfrecord')
pipeline_config_path = 'C:/ssd_mobilenet_v2_320x320_coco17_tpu-8/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config'
model_dir = 'C:/AI_hub/model'

# 데이터셋 로드 함수
def load_dataset(tfrecord_file, batch_size):
    feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/source_id': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }

    def parse_example(example_proto):
        try:
            return tf.io.parse_single_example(example_proto, feature_description)
        except tf.errors.DataLossError:
            raise ValueError("Corrupted record encountered. Please remove the corrupted record and try again.")

    def decode_image(parsed_example):
        image = tf.image.decode_image(parsed_example['image/encoded'], channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        return image

    def process_data(parsed_example):
        image = decode_image(parsed_example)
        labels = parsed_example['image/object/class/label']
        return image, labels

    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.filter(lambda x: x is not None)
    dataset = dataset.map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()

    return dataset, dataset_size

# 모델 구성 로드
pipeline_config = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = pipeline_config['model']
detection_model = model_builder.build(model_config=model_config, is_training=True)

# 모델 빌드
@tf.function
def build_model(input):
    detection_model(input)

# 빈 입력으로 모델 빌드
dummy_input = tf.ones((1, 320, 320, 3))
build_model(dummy_input)

# 옵티마이저 설정
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# 체크포인트 설정
checkpoint_dir = 'C:/AI_hub/checkpoints'
checkpoint = tf.train.Checkpoint(model=detection_model, optimizer=optimizer)
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

# 최신 체크포인트를 로드하고, 상태를 복구
if latest_checkpoint:
    checkpoint.restore(latest_checkpoint)
    print("Loaded checkpoint:", latest_checkpoint)

# 손상된 파일 확인 함수
def is_file_corrupted(tfrecord_file):
    try:
        with open(tfrecord_file, 'rb') as file:
            file.read()
        return False
    except:
        return True

# 손상된 파일이 있다면 경고 메시지 출력
if is_file_corrupted(training_tfrecord_file):
    print("TFRecord 파일이 손상되었습니다. 파일을 다시 생성해주세요.")
else:
    # 배치 사이즈 설정
    batch_size = 32

    # 데이터셋 로드
    training_dataset, training_size = load_dataset(training_tfrecord_file, batch_size)
    validation_dataset, validation_size = load_dataset(validation_tfrecord_file, batch_size)

    # 훈련 시작
    for epoch in range(10):
        for inputs, labels in tqdm(training_dataset):
            with tf.GradientTape() as tape:
                predictions = detection_model(inputs)
                loss = compute_loss(labels, predictions)
            gradients = tape.gradient(loss, detection_model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, detection_model.trainable_variables))
        print("Epoch", epoch, "finished")

        # 검증 시작
        for inputs, labels in tqdm(validation_dataset):
            predictions = detection_model(inputs)
            loss = compute_loss(labels, predictions)
        print("Validation loss:", loss)


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


...

INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


TFRecord 파일이 손상되었습니다. 파일을 다시 생성해주세요.


In [ ]:
import tensorflow as tf
from object_detection.utils import dataset_util, config_util
from object_detection.builders import model_builder
from object_detection.core import losses
import os
import s3fs
import time

num_classes = 98

# AWS S3 액세스 키 및 시크릿 액세스 키 설정
access_key = 'access_key'
secret_key = 'secret_key'

# S3 파일 시스템 초기화
print("S3 파일 시스템 초기화")
fs = s3fs.S3FileSystem(key=access_key, secret=secret_key)

# 버킷 설정
source_bucket_name = 'tfrecord-bucket02'
destination_bucket_name = 'module-bucket01'

# 손실 함수 정의
print("손실 함수 정의")
binary_crossentropy = tf.keras.losses.BinaryCrossentropy()

def compute_loss(labels, predictions):
    print("compute_loss")
    localization_loss = binary_crossentropy(labels['groundtruth_boxes'], predictions['predicted_boxes'])
    classification_loss = binary_crossentropy(labels['groundtruth_classes'], predictions['predicted_classes'])
    return {'localization_loss': localization_loss, 'classification_loss': classification_loss}

# 경로 설정
training_tfrecord_file = 's3://{}/training.tfrecord'.format(source_bucket_name)
validation_tfrecord_file = 's3://{}/validation.tfrecord'.format(source_bucket_name)
pipeline_config_path = 'C:/ssd_mobilenet_v2_320x320_coco17_tpu-8/ssd_mobilenet_v2_320x320_coco17_tpu-8/pipeline.config'
model_dir = 'C:/AI_hub/model'

# pipeline.config 파일 열기
with open(pipeline_config_path, 'r') as f:
    print("pipeline.config 파일 열기")
    config_data = f.read()

# input_path 수정
print("input_path 수정")
config_data = config_data.replace('input_path: "s3://tfrecord-bucket02/validation.tfrecord""', 'input_path: "{}"'.format(validation_tfrecord_file))

# 수정된 내용으로 pipeline.config 파일 저장
with open(pipeline_config_path, 'w', newline='\n') as f:
    print("수정된 내용으로 pipeline.config 파일 저장")
    f.write(config_data)

# 데이터셋 로드 함수
def load_dataset(tfrecord_file, batch_size):
    print("데이터셋 로드 함수")
    feature_description = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/source_id': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }

    def parse_example(example_proto):
        print("parse_example 함수")
        try:
            return tf.io.parse_single_example(example_proto, feature_description)
        except tf.errors.DataLossError:
            raise ValueError("Corrupted record encountered. Please remove the corrupted record and try again.")

    def decode_image(parsed_example):
        print("decode_image 함수")
        image = tf.image.decode_image(parsed_example['image/encoded'], channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        return image

    def process_data(parsed_example):
        print("process_data 함수")
        image = decode_image(parsed_example)
        boxes = tf.stack([
            tf.sparse.to_dense(parsed_example['image/object/bbox/xmin']),
            tf.sparse.to_dense(parsed_example['image/object/bbox/ymin']),
            tf.sparse.to_dense(parsed_example['image/object/bbox/xmax']),
            tf.sparse.to_dense(parsed_example['image/object/bbox/ymax'])
        ], axis=-1)

        classes = tf.sparse.to_dense(parsed_example['image/object/class/label'])
        classes = tf.one_hot(classes, depth=num_classes)
        labels = {'groundtruth_boxes': boxes, 'groundtruth_classes': classes}
        return image, labels

    def is_file_corrupted(file_path):
        print("is_file_corrupted 함수")
        try:
            with fs.open(file_path, 'rb') as file:
                file.read()
            return False
        except:
            return True

    if is_file_corrupted(tfrecord_file):
        raise ValueError("TFRecord 파일이 손상되었습니다. 파일을 다시 생성해주세요.")
        
    print("dataset")
    dataset = tf.data.TFRecordDataset(tfrecord_file, compression_type='GZIP')
    dataset = dataset.map(parse_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.filter(lambda x: x is not None)
    dataset = dataset.map(process_data, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    dataset_size = tf.data.experimental.cardinality(dataset).numpy()

    return dataset, dataset_size

# 모델 구성 로드
print("모델 구성 로드")
pipeline_config = config_util.get_configs_from_pipeline_file(pipeline_config_path)
model_config = pipeline_config['model']
detection_model = model_builder.build(model_config=model_config, is_training=True)

# 모델 빌드
@tf.function
def build_model(input):
    print("모델 빌드")
    detection_model(input)

# 빈 입력으로 모델 빌드
print("빈 입력으로 모델 빌드")
dummy_input = tf.ones((1, 320, 320, 3))
build_model(dummy_input)

# 옵티마이저 설정
print("옵티마이저 설정")
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# 체크포인트 설정
print("체크포인트 설정")
checkpoint_dir = 'C:/AI_hub/checkpoints'
checkpoint = tf.train.Checkpoint(model=detection_model, optimizer=optimizer)
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)

# 최신 체크포인트를 로드하고, 상태를 복구
if latest_checkpoint:
    print("최신 체크포인트를 로드하고, 상태를 복구")
    checkpoint.restore(latest_checkpoint)
    print("Loaded checkpoint:", latest_checkpoint)

# 훈련 및 검증 시작
def train_and_evaluate(training_dataset, validation_dataset, num_epochs):
    print("훈련 및 검증 시작")
    for epoch in range(num_epochs):
        print("1")
        start_time = time.time()  # 학습 시작 시간 기록
        for inputs, labels in training_dataset:
            with tf.GradientTape() as tape:
                print("2")
                preprocessed_images = tf.image.resize(inputs, (320, 320))
                print("3")
                prediction_dict = detection_model.predict(preprocessed_images)
                print("4")
                losses_dict = compute_loss(labels, prediction_dict)
                print("5")
                total_loss = losses_dict['localization_loss'] + losses_dict['classification_loss']
                print("6")
            print("7")
            gradients = tape.gradient(total_loss, detection_model.trainable_variables)
            print("8")
            optimizer.apply_gradients(zip(gradients, detection_model.trainable_variables))
        print("Epoch", epoch, "finished")
        end_time = time.time()  # 학습 종료 시간 기록
        elapsed_time = end_time - start_time  # 학습 시간 계산
        print("Elapsed time: {:.2f} seconds".format(elapsed_time))

        validation_loss = 0.0
        num_batches = 0
        for inputs, labels in validation_dataset:
            preprocessed_images = tf.image.resize(inputs, (320, 320))
            prediction_dict = detection_model.predict(preprocessed_images)
            losses_dict = compute_loss(labels, prediction_dict)
            validation_loss += losses_dict['localization_loss'] + losses_dict['classification_loss']
            num_batches += 1
        validation_loss /= num_batches
        print("Validation loss:", validation_loss)

# 배치 사이즈 설정
batch_size = 32

# 데이터셋 로드
try:
    print("데이터셋 로드")
    training_dataset, training_size = load_dataset(training_tfrecord_file, batch_size)
    validation_dataset, validation_size = load_dataset(validation_tfrecord_file, batch_size)

    # 훈련 및 검증 시작
    train_and_evaluate(training_dataset, validation_dataset, num_epochs=10)
except Exception as e:
    print("Failed to load dataset:", str(e))

# 학습이 완료된 모델을 저장
model_dir = 'C:/AI_hub/model'
checkpoint.save(file_prefix=os.path.join(model_dir, 'ckpt'))

# 모델을 S3에 업로드
model_files = [
    os.path.join(model_dir, 'ckpt.data-00000-of-00001'),
    os.path.join(model_dir, 'ckpt.index'),
    os.path.join(model_dir, 'ckpt.meta')
]

for model_file in model_files:
    destination_path = 's3://{}/{}'.format(destination_bucket_name, os.path.basename(model_file))
    try:
        fs.put(model_file, destination_path)
        print("Uploaded model file:", destination_path)
    except Exception as e:
        print("Failed to upload model file:", str(e))


S3 파일 시스템 초기화
손실 함수 정의
pipeline.config 파일 열기
input_path 수정
수정된 내용으로 pipeline.config 파일 저장
모델 구성 로드
빈 입력으로 모델 빌드
모델 빌드
INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


...

INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


INFO:tensorflow:depth of additional conv before box predictor: 0


모델 빌드
옵티마이저 설정
체크포인트 설정
데이터셋 로드
데이터셋 로드 함수
is_file_corrupted 함수
dataset
parse_example 함수
process_data 함수
decode_image 함수
데이터셋 로드 함수
is_file_corrupted 함수
dataset
parse_example 함수
process_data 함수
decode_image 함수
훈련 및 검증 시작
1
